In [221]:
import requests
import lxml.html as lh
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)



In [222]:
tr_elements =doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [223]:
for j in range(1, len(tr_elements)):
    T = tr_elements[j]
    if len(T)!=3:
        break
       
    i=0
    for t in T.iterchildren():
        data=t.text_content()
        if i>0:
              try:
                data =int(data)
              except:
                 pass
        col[i][1].append(data)
        i+=1
#[len(C) for (title, C) in col] 


In [224]:
Dict={title:column for (title,column) in col}


df=pd.DataFrame(Dict)
df.head()

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [225]:
cols=df.columns.tolist()
cols


['Borough', 'Neighbourhood\n', 'Postcode']

In [226]:
cols = cols[-1:] + cols[:-1]
cols

['Postcode', 'Borough', 'Neighbourhood\n']

In [227]:
df=df[cols]
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [228]:
import numpy as np

df.replace("Not assigned", np.nan, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned\n
1,M2A,NaN,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [229]:
missing_data =df.isnull()
missing_data.head()

,Postcode,Borough,Neighbourhood
0,False,True,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False


In [230]:
df.dropna(subset=["Borough"],axis=0, inplace=True)

In [231]:
df.reset_index(drop=True, inplace=True)

In [287]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [288]:
df_postcode=df.groupby('Postcode')  
df_postcode.groups

{'M1B': Int64Index([8, 9], dtype='int64'),
 'M1C': Int64Index([21, 22, 23], dtype='int64'),
 'M1E': Int64Index([33, 34, 35], dtype='int64'),
 'M1G': Int64Index([39], dtype='int64'),
 'M1H': Int64Index([43], dtype='int64'),
 'M1J': Int64Index([54], dtype='int64'),
 'M1K': Int64Index([66, 67, 68], dtype='int64'),
 'M1L': Int64Index([79, 80, 81], dtype='int64'),
 'M1M': Int64Index([93, 94, 95], dtype='int64'),
 'M1N': Int64Index([108, 109], dtype='int64'),
 'M1P': Int64Index([117, 118, 119], dtype='int64'),
 'M1R': Int64Index([127, 128], dtype='int64'),
 'M1S': Int64Index([141], dtype='int64'),
 'M1T': Int64Index([147, 148, 149], dtype='int64'),
 'M1V': Int64Index([155, 156, 157, 158], dtype='int64'),
 'M1W': Int64Index([182], dtype='int64'),
 'M1X': Int64Index([188], dtype='int64'),
 'M2H': Int64Index([44], dtype='int64'),
 'M2J': Int64Index([55, 56, 57], dtype='int64'),
 'M2K': Int64Index([69], dtype='int64'),
 'M2L': Int64Index([82, 83], dtype='int64'),
 'M2M': Int64Index([96, 97], dty